# Base de dados

Baixem a base de dados de [recorded mundiais](https://vincentarelbundock.github.io/Rdatasets/csv/DAAG/worldRecords.csv) e coloquem o arquivo na mesma pasta que este notebook

Vamos utilizar [pandas](https://pandas.pydata.org/) para processamento de dados e [matplotlib](https://matplotlib.org/) para plotar gráficos


In [1]:
# awesome code

# Análise de dados

Usando sqlite3 e scikit-learn.

Desafio: construa uma heurística que receba como input os atributos da flores iris e retorne a espécie da iris.

In [ ]:
# awesome analysis